# Today's session

In today's session, we will get to know some of the genomics tools that were introduced in the theoretical lectures. In particular, we will see how to work with sequences, align them, and so on. To do this, we will be using `Biopyton`, a comprehensive Python module that allows you to do these and many other things. For a complete tour of `Biopython`, take a look at the [complete tutorial](http://biopython.org/DIST/docs/tutorial/Tutorial.html).

In the final exercise for this class you will be working on the following case study:

>A few weeks ago, a patient was admitted to the hospital where you work. The patient was coughing, and had difficulty breathing, as well as a fever. Your colleagues Oscar and Maria, from the genomics platform at the hospital, have been able to isolate a viral genome from the patient; this virus is thought to be responsible for the disease. You are expected to identify what the virus is and, thus, provide a diagnosis. You are also expected to compare the virus' genome with the genome of similar viruses to try to understand where the disease came from and how is it different from known diseases. 

# What is Biopython?

The Biopython Project is an international association of developers of freely available Python tools for computational molecular biology.

![Biopython](Media/biopython_logo.png)


The main Biopython releases have lots of functionality, including:

* The ability to parse bioinformatics files into Python utilizable data structures, including support for the following formats:
    - Blast output – both from standalone and WWW Blast
    - Clustalw
    - FASTA
    - GenBank
    - PubMed and Medline
    - ExPASy files, like Enzyme and Prosite
    - SCOP, including ‘dom’ and ‘lin’ files
    - UniGene
    - SwissProt 
* Files in the supported formats can be iterated over record by record or indexed and accessed via a Dictionary interface.
* Code to deal with popular on-line bioinformatics destinations such as:
    - NCBI – Blast, Entrez and PubMed services
    - ExPASy – Swiss-Prot and Prosite entries, as well as Prosite searches 
* Interfaces to common bioinformatics programs such as:
    - Standalone Blast from NCBI
    - Clustalw alignment program
    - EMBOSS command line tools 
* A standard sequence class that deals with sequences, ids on sequences, and sequence features.
* Tools for performing common operations on sequences, such as translation, transcription and weight calculations.
* Code for dealing with alignments, including a standard way to create and deal with substitution matrices.
* Code making it easy to split up parallelizable tasks into separate processes.
* GUI-based programs to do basic sequence manipulations, translations, BLASTing, etc.
* Extensive documentation and help with using the modules, including the tutorial, on-line wiki documentation, the web site, and the mailing list.
* Integration with BioSQL, a sequence database schema also supported by the BioPerl and BioJava projects.

# Sequences in `Biopython`

We will start with a quick introduction to the `Biopython` class for dealing with sequences. Most of the time, when we think of sequences what we have in my mind is a string of letters like 'AGTACACTGGT'. In `Biopython` you can create a `Seq` instance from such a sequence as follows:

In [ ]:
from Bio.Seq import Seq

my_seq = Seq("AGTACACTGGT")
display(my_seq)

## Sequence alphabet

The sequence above seems to represent a fragment of DNA (that is, a sequence of bases), but it could also be a protein, since A, G, T, and C could also represent aminoacids. To specify the type of sequence we are dealing with, the `Seq` class has an `alphabet` attribute. Let's see how it works:

In [ ]:
from Bio.Alphabet import IUPAC

my_seq = Seq("AGTACACTGGT", alphabet=IUPAC.unambiguous_dna)
display(my_seq)
display(my_seq.alphabet)

## Transcription and  translation

So, *now* we know that we are dealing with a DNA sequence. And, in fact, we can do things with this sequence that we could not do with a sequence of aminoacids. For example, assuming that the `coding_dna` sequence corresponds to the 5'-to-3′ strand of the DNA, we can obtain the corresponding 5'-to-3' complementary strand:

In [ ]:
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", IUPAC.unambiguous_dna)
display(coding_dna.reverse_complement())

Or, we can **transcribe** this DNA sequence into an RNA sequence:

In [ ]:
messenger_rna = coding_dna.transcribe()
display(messenger_rna)

Translation is a bit tricker, because codons are translated differently in different organisms and even different cellular compartments. Here is what happens if we translate and RNA sequence without any parameters:

In [ ]:
messenger_rna.translate()

You should notice in the above protein sequence that, in addition to the end stop character, there is an internal stop as well. This was a deliberate choice of example, as it gives an excuse to talk about some optional arguments, including different translation tables (genetic codes).

The translation tables available in Biopython are based on those from [the NCBI](https://www.ncbi.nlm.nih.gov/Taxonomy/Utils/wprintgc.cgi). By default, translation will use the standard genetic code (NCBI table id 1). But suppose we are dealing with a mitochondrial sequence---we need to tell the translation function to use the relevant genetic code instead:

In [ ]:
messenger_rna.translate(table="Vertebrate Mitochondrial")

## Otherwise, sequences are similar to strings

Other than the situations described above, most of the time `Seq` instances can be dealt with just like strings. For example, we can extract the elements of a sequence using indices, or slice a sequence as we would slice a string: 

In [ ]:
# Seq instances work, mostly, as strings
my_seq = Seq("GATCGATGGGCCTATATAGGATCGAAAATCGC", IUPAC.unambiguous_dna)
display(my_seq[4])
display(my_seq[4:12])
print('Length of the sequence:      ', len(my_seq))
print('Number of As in the sequence:', my_seq.count('A'))

# Reading and writing annotated sequence files

Typically, we do not want to create `Seq` instances by entering manually the sequence of letters (bases or aminoacids). Rather, we usually want to read a bunch of annotated sequences from a file or from some remote database. By annotated sequence here we mean a sequence to which additional information has been added (for example, about the organism, the positions of coding sequences within the sequence, etc.). Here we will learn how to read (and write) annotated sequences from (and to) files. First, we need to import the input/output functions in `Biopython`:  

In [ ]:
from Bio import SeqIO

## Reading GeneBank

Now, we are going to read the file `Files/ls_orchid.gb`, which contains several annotated sequences in GeneBank format. But before we read it using `Biopython`...

***
### Exercise

Open the file `Files/ls_orchid.gb` using a text editor such as Notepad or Word. Familiarize yourself with the format of the file. Copy the first complete annotated record in the file and paste it in the cell below.

*Write your answer here.*

***

Now, let's do something similar but using `Biopython` and the `SeqIO.parse()` method:

In [ ]:
# Read the GeneBank file and create a list with the records
records = list(SeqIO.parse('Files/ls_orchid.gb', 'genbank'))

print('The file comprises', len(records), 'records')

Let us now select the first record and print it:

In [ ]:
print(records[0])

The `SeqRecord` class that holds annotated sequences has the following information as attributes:

**.seq**
– The sequence itself, typically a Seq object.

**.id**
– The primary ID used to identify the sequence – a string. In most cases this is something like an accession number.

**.name**
– A “common” name/id for the sequence – a string. In some cases this will be the same as the accession number, but it could also be a clone name. I think of this as being analogous to the LOCUS id in a GenBank record.

**.description**
– A human readable description or expressive name for the sequence – a string.

**.letter_annotations**
– Holds per-letter-annotations using a (restricted) dictionary of additional information about the letters in the sequence. The keys are the name of the information, and the information is contained in the value as a Python sequence (i.e. a list, tuple or string) with the same length as the sequence itself. This is often used for quality scores or secondary structure information.

**.annotations**
– A dictionary of additional information about the sequence. The keys are the name of the information, and the information is contained in the value. This allows the addition of more “unstructured” information to the sequence.

**.features**
– A list of SeqFeature objects with more structured information about the features on a sequence (e.g. position of genes on a genome, or domains on a protein sequence).

**.dbxrefs** - A list of database cross-references as strings.

For example:

In [ ]:
# Print some of the attributes of the first annotated record in the file
print('SEQUENCE:', records[0].seq, '\n')
print('SEQUENCE LENGTH:', len(records[0].seq), '\n')
print('FEATURES:')
for feature in records[0].features:
    print(feature)

***
### Exercise

Select the record with the longest sequence in the file, and print its features. Then, explain in words what each feature corresponds to.

In [ ]:
# Write your code here

*Explain succinctly each feature*

***

## Reading FASTA

GeneBank annotated records are quite complete. Now compare with the annotation in typical FASTA files:

In [ ]:
# Read the GeneBank file and create a list with the records
frecords = list(SeqIO.parse('Files/ls_orchid.fasta', 'fasta'))

print('The file comprises', len(frecords), 'records')
print(frecords[0])

The resulting `SeqRecord` is the same as before, but the annotation is much poorer than in a typical GeneBank file.

## Writing annotated sequence files

Finally, to write a bunch of annotated sequences to a file we can use the `write()` method. For example, to write the first 5 records of the GeneBank file we read above to a file named `Files/five_records.fasta` in FASTA format, we need to do:

In [ ]:
SeqIO.write(records[:5], 'Files/five_records.fasta', 'fasta')

***

### Exercise

Using `Biopython`, open the `Files/five_records.fasta` file that you just created and verify that it contains 5 records.

In [ ]:
# Write your code here

### Exercise

Using `Biopython`, write the first 10 records in `records` to a file named `Files/ten_records.gb` in GeneBank format.

In [ ]:
# Write your code here

***

# Getting annotated sequences from online databases

We have seen how to read local GeneBank or FASTA files. Therefore, at this point we should be able to go online, manually obtain whatever files we need, and then read them using `Biopython`. If we need to get many files, this is impractical. Here, we will see how to get annotated sequence files **from online databases directly**, that is, without having to manually download them.

`Biopython` provides interfaces for several online databases. However, here we will focus on obtaining annotaded records for the [Entrez database](https://www.ncbi.nlm.nih.gov/search/). So, first, we import the Entrez functionalities in `Biopython`.

In [ ]:
from Bio import Entrez

The `.efetch()` method allows us to "fetch" entries from the database. Let's first see how to use it in practice:

In [ ]:
# First, we need to identify ourselves to Entrez
Entrez.email = "name.surname@urv.cat" # Modify with your actual account

# Fetch one record by its ID (Z78502.1) and store it in the entrez_recs list of SeqRecord instances
with Entrez.efetch(db="nucleotide", id='Z78502.1', rettype="gb", retmode="text") as my_handle:
    entrez_recs = list(SeqIO.parse(my_handle, "genbank"))

# Show some stuff    
print(len(entrez_recs), 'record(s) read\n')
print(entrez_recs[0])

The `efetch()` method needs several arguments. First, we need to specify which database, within Entrez, are we interested in. In this case, we want to work with the `nucleotide` database. Second, we need to pass the ID of the entry we are interested in, in this case Z78502.1. Finally, we need to specify the format of the annotated record ('gb' for GeneBank in this case, but it could be FASTA or others), and the format of the output (text).

Importantly, the `.efetch()` method does *not* return a list of `SeqRecord` instances directly. Rather, **`.efetch()` returns a "handle"**, which can be understood as a file that is open for reading. We store this handle in the variable `my_handle` and **then parsed with `SeqIO.parse()`**, just as we read annotated sequences from files earlier.

We can also get several annotated sequences in a single search as follows:

In [ ]:
# Fetch multiple records by their ID and store them in the entrez_recs list of SeqRecord instances.
# Note that the multiples IDs are passed as a single string of comma-separated IDs, rather than a list of IDs.
with Entrez.efetch(db="nucleotide", id='AF191665.1, AF191664.1, AF191663.1', rettype="gb", retmode="text") as my_handle:
    entrez_recs = list(SeqIO.parse(my_handle, "genbank"))
print(len(entrez_recs), 'record(s) read\n')
print(entrez_recs[0])

OK, this is a good point to introduce a useful trick for learning more about a given module, function, class or whatever: For mor information about `.efetch()` (and, similarly, for other methods) you can run the following:

In [ ]:
help(Entrez.efetch)

# Sequence alingnments and BLAST

So far, we have learned how to manage sequences and annotated sequences, including reading and writing files and obtaining information from remote databases. Last, we turn to the problem of sequence alignment. We start by considering local and global pairwise alignments (alignments of two sequences), and then turn to sequence search in databases using BLAST.

## Pairwise alignments

The `Bio.Align.PairwiseAligner` implements the Needleman-Wunsch, Smith-Waterman, Gotoh (three-state), and Waterman-Smith-Beyer global and local pairwise alignment algorithms. Let's see a simple example

In [ ]:
# Import Align
from Bio import Align

# Define the sequences we intend to align
seq1 = Seq("GAACTAAATTCGTA", alphabet=IUPAC.ambiguous_dna)
seq2 = Seq("GATAAATAGTA", alphabet=IUPAC.ambiguous_dna)

# Create the pairwise aligner, which you will use to make the alignment
aligner = Align.PairwiseAligner()

# Align the two sequences
alignments = aligner.align(seq1, seq2)

# Show results
for alignment in alignments:
    print('SCORE:', alignment.score)
    print(alignment)

The aligner object (an instance of the `PairwiseAligner` class) is the "machine" that will allow you to do the alignment. When you create a `PairwiseAligner` it takes some default values for things like the algorithm it uses, the scores for match, mismatch, or gap, or whether the alignment is local/global. Below we will learn how to change these default values.

After the `PairwiseAligner` has been created, is just a matter or getting the alignments using the `.align()` method. Note that the output of `.align()` is not a single alignment, but a bunch of alignments with high score.

***

### Exercise

Use the `help()` function (as we did for `Entrez.efetch()` above) to learn which are the attributes of the `PairwiseAlignment` class storing: the algorithm being used by the aligner we just created, the match score, the mismatch score, the gap score, and whether the alignment is local/global. 

In [ ]:
# Use help here

In [89]:
# Write your code here to print the algorithm, the scores, and whether the alignment is local/global. 

*** 

Any of the attributes of the `PairwiseAligner` can be changed once an instance has been created and before actually aligning the sequences. For example, we can change the scores and, of course, we will get (slightly) different alignments: 

In [ ]:
# Create the pairwise aligner
aligner2 = Align.PairwiseAligner()

# Change substitution scores
aligner2.match_score = 1.0
aligner2.mismatch_score = -2.0
aligner2.gap_score = -2.5

# Align the two sequences and show results
alignments2 = aligner2.align(seq1, seq2)
for alignment in alignments2:
    print('SCORE:', alignment.score)
    print(alignment)

## BLAST

Pairwise aligments are useful when 

# sgsgsgsg

In [ ]:
from Bio.Blast import NCBIWWW

In [ ]:
help(NCBIWWW.qblast)

In [ ]:
%time result_handle = NCBIWWW.qblast("blastn", "nt", record_dict['MN908947.3'].seq)
with open("Files/my_blast.xml", "w") as out_handle:
    out_handle.write(result_handle.read())


In [ ]:
from Bio.Blast import NCBIXML

In [ ]:
result_handle = open("Files/my_blast.xml")
blast_records = list(NCBIXML.parse(result_handle))

![UML diagram of the BlastRecord](Media/BlastRecord.png)

In [ ]:
E_VALUE_THRESH = 0.01
print(len(blast_records[0].alignments))
for alignment in blast_records[0].alignments:
    for hsp in alignment.hsps:  # HSPs = high-scoring pairs
        if hsp.expect < E_VALUE_THRESH:
            print("\n****Alignment****")
            print("sequence  :", alignment.title)
            print("length    :", alignment.length)
            print("e value   :", hsp.expect)
            print("identities:", hsp.identities)
            print(hsp.query[0:75] + "...")
            print(hsp.match[0:75] + "...")
            print(hsp.sbjct[0:75] + "...")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_alignment(alma, thin=50):
    plt.figure(figsize=(20, 5))
    nmatch = 0
    for i in range(len(alma)):
        if alma[i] == '|':
            nmatch += 1
        if ((i+1) % thin) == 0:
            plt.plot((i, i), (0, 1), c='g', alpha=((nmatch - 0.25)/thin))
            nmatch = 0
    plt.show()
    return

def plot_alignment2(alma, thin=500):
    plt.figure(figsize=(20, 5))
    nmatch, x, y = 0, [], []
    for i in range(len(alma)):
        if alma[i] == '|':
            nmatch += 1
        if ((i+1) % thin) == 0:
            x.append(i)
            y.append(nmatch/thin)           
            nmatch = 0
    plt.plot(x, y)
    plt.show()
    return  

In [ ]:
als2 = str(alignments[0])[:len(str(alignments[0])) // 3]
alma = str(alignments[0])[len(str(alignments[0])) // 3 : 2 * len(str(alignments[0])) // 3]
als1 = str(alignments[0])[2 * len(str(alignments[0])) // 3:]

print(als1[:75])
print(alma[:75])
print(als2[:75])

In [ ]:
alma.count('|') / len(alma)

In [ ]:
plot_alignment(alma[:])

In [ ]:
plot_alignment2(alma[:])

In [ ]:
print(alignments[0].query[:75])
print(alignments[0].target[:75])
print(alignments[0].score)
#print(alignments[0].path)
#print(alignments[0].aligned)
dir(alignments[0])


# Final exercise (5/10 points): The misterious virus

A few weeks ago, a patient was admitted to the hospital where you work. The patient was coughing, and had difficulty breathing, as well as a fever. Your colleagues Oscar and Maria, from the genomics platform at the hospital, have been able to isolate a viral genome from the patient; this virus is thought to be responsible for the disease. **You are expected to identify what the virus is and, thus, provide a diagnosis**. You are also expected to compare the virus' genome with the genome of similar viruses to try to **understand where the disease came from and how is it different from closely-related disease-causing viruses**. 

1. **Open and read genome of the virus** using Biopython. The genome is available as a FASTA file from `Files/patient.fasta`). 
2. Go to the BLAST [web page](https://blast.ncbi.nlm.nih.gov/Blast.cgi) and **BLAST the sequence manually**. BLAST against the standard nucleotide collections using the best algorithm for the alignment of highly similar sequences. After blasting, obtain, show (as a screenshot), and discuss the following visualizations:
 - A **graphic summary of the alignments**. 
 - A **distance tree** of the results.


3. BLAST the sequence again, **using Biopython** this time. Be aware that this can take several minutes to complete (~10 minutes), so be patient. Upon completion of the BLAST search, immediately **save the BLAST results in a file** named `Files/blast_results.xml`.
4. Open the results and **display all alignments with percentage identity larger than 99.5%**. Which is your diagnosis for the patient?
5. **Display all alignments with percentage identity larger than 85.0% and smaller than 99.5%**. Which animal do you think may have transmitted the disease to humans?
6. Choose one of the sequences with percentage similarity between 85% and 90%. Get the whole **GeneBank entry** for this sequence **from Entrez using Biopython**, and display it. List, also, the coding regions of this genome indicating start and end of each coding sequence (CDS).
7. Do a **global pairwise alignment** between the genome in question 6 and the patient's viral genome using the **Needleman-Wunsch algorithm in Biopython**. Visualize the aligment using the `plot_alignment()` function defined above, and explain in which region of the original virus (question 6) has the new virus mutated. For extra credit, write a new `plot_alignment_new()` function that helps you address this question  more precisely (e.g. by marking the start and end of CDSs in the genome of question 6).

In [ ]:
# Question 1 code here

![Save the graphic summary of the alignments as Files/blast_align.png and, after Ctrl+Enter in this cell, the image should show up here!](Files/blast_align.png)

![Save the distance tree of the alignments as Files/blast_tree.png and, after Ctrl+Enter in this cell, the image should show up here!](Files/blast_tree.png)

*Discuss the results here*

In [ ]:
# Question 3 code here

In [ ]:
# Question 4 code here

*Answer to question 4 here*

In [ ]:
# Question 5 code here

*Answer to question 5 here*

In [ ]:
# Question 6 code here

In [ ]:
# Question 7 code here

*Answer to question 7 here*